## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Descubrimiento de grupos en los datos

---

En esta libreta se generan los modelos de clusterización sobre el dataset final del escenario para su posterior evaluación y análisis. Se utilizará como herramienta de soporte a mlflow para el registro completo de la experimentación. 

---

In [63]:
# Importación de librerías

import pandas as pd
# Se importa mlflow para registro de la experimentación 
import mlflow
# Se importan los métodos a utilizar para clusterizar
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering, MeanShift, Birch
# Se importan las métricas a utilizar para evaluar el proceso
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score


----

##### Lectura del dataset

In [64]:
df = pd.read_csv("../../../data/final/datos_clusterizacion.csv", sep=";")
df.head(5)

,edad,importe_solicitado,duracion_credito,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,situacion_vivienda_ALQUILER,...,estado_civil_CASADO,estado_civil_DESCONOCIDO,estado_civil_DIVORCIADO,estado_civil_SOLTERO,estado_cliente_ACTIVO,estado_cliente_PASIVO,nivel_tarjeta_Blue,nivel_tarjeta_Gold,nivel_tarjeta_Platinum,nivel_tarjeta_Silver
0,22,35000,3,59000,0.59,16.02,1,36.0,1088.0,True,...,True,False,False,False,True,False,True,False,False,False
1,21,1000,2,9600,0.10,11.14,0,39.0,1144.0,False,...,True,False,False,False,True,False,True,False,False,False
2,23,35000,2,65500,0.53,15.23,1,36.0,1887.0,True,...,True,False,False,False,True,False,True,False,False,False
3,24,35000,4,54400,0.55,14.27,1,54.0,1314.0,True,...,True,False,False,False,True,False,True,False,False,False
4,21,2500,2,9900,0.25,7.14,1,34.0,1171.0,False,...,False,True,False,False,True,False,True,False,False,False


----

##### Configuración de la experimentación

In [65]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering 1'
exp_id = mlflow.create_experiment(name=exp_name)

- Kmeans

In [66]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K2")

    # Se finaliza el registro
    mlflow.end_run()

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [67]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k2 = df.copy()
# Se agregan las labels generadas
df_kmeans_k2['cluster'] = cluster_labels
df_kmeans_k2['cluster'].value_counts()

cluster
0    9434
1     668
Name: count, dtype: int64

In [68]:
df_kmeans_k2.to_csv("../../../data/final/resultados_kmeans_k2.csv", sep=";", index=False)

Se pasa a probar otro método (clustering jerárquico)

- AgglomerativeClustering

In [69]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K3")

    # Se finaliza el registro
    mlflow.end_run()

2024/05/05 16:27:17 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [70]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k3 = df.copy()
# Se agregan las labels generadas
df_agc_k3['cluster'] = cluster_labels
df_agc_k3['cluster'].value_counts()

cluster
0    5070
2    4861
1     171
Name: count, dtype: int64

Finalmente, se prueba un método que no tiene parámetros

- MeanShift

In [71]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="MeanShift"):
    # Selección y configuración de la técnica
    modelo_clusters = MeanShift()
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "MS")

    # Se finaliza el registro
    mlflow.end_run()

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [72]:
# Se preserva el conjunto de datos original para evitar problemas
df_ms = df.copy()
# Se agregan las labels generadas
df_ms['cluster'] = cluster_labels
df_ms['cluster'].value_counts()

cluster
0    9536
1     407
2      93
3      47
4       7
5       6
6       5
7       1
Name: count, dtype: int64

- Birch

Decido realizar otro método más, en este caso Birch (algoritmo de clustering basado en árboles). Para esta primera prueba indico un threshold de 0.5 y un numero de clusters igual a 2.

In [73]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Birch - threshold=0.5 - n_clusters=2"):

    # Selección y configuración de la técnica
    modelo_clusters = Birch(threshold=0.5, n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro threshold
    mlflow.log_param('Valor threshold', 0.5)
    # Se registra el parámetro num clusters
    mlflow.log_param('Valor n_clusters', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Birch_0.5/2")

    # Se finaliza el registro
    mlflow.end_run()

In [74]:
# Se preserva el conjunto de datos original para evitar problemas
df_birch2 = df.copy()
# Se agregan las labels generadas
df_birch2['cluster'] = cluster_labels
df_birch2['cluster'].value_counts()

cluster
0    9931
1     171
Name: count, dtype: int64

In [75]:
df_birch2.to_csv("../../../data/final/resultados_birch_n2.csv", sep=";", index=False)

----

Prueba Nro. 2

- Kmeans


In [76]:
# Se genera el experimento en mlflow
exp_name_2 = 'Experimentación clustering 2'
exp_id_2 = mlflow.create_experiment(name=exp_name_2)

In [77]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id_2, run_name="Kmeans - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3")

    # Se finaliza el registro
    mlflow.end_run()

In [78]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k3 = df.copy()
# Se agregan las labels generadas
df_kmeans_k3['cluster'] = cluster_labels
df_kmeans_k3['cluster'].value_counts()

cluster
0    5199
2    4534
1     369
Name: count, dtype: int64

In [79]:
df_kmeans_k3.to_csv("../../../data/final/resultados_kmeans_k3.csv", sep=";", index=False)

- AgglomerativeClustering

In [80]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id_2, run_name="AGClustering - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K2")

    # Se finaliza el registro
    mlflow.end_run()

2024/05/05 16:40:50 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [81]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
0    9931
1     171
Name: count, dtype: int64

- Birch

In [82]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id_2, run_name="Birch - threshold=0.5 - n_clusters=3"):

    # Selección y configuración de la técnica
    modelo_clusters = Birch(threshold=0.5, n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro threshold
    mlflow.log_param('Valor threshold', 0.5)
    # Se registra el parámetro num clusters
    mlflow.log_param('Valor n_clusters', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Birch_0.5/3")

    # Se finaliza el registro
    mlflow.end_run()

In [83]:
# Se preserva el conjunto de datos original para evitar problemas
df_birch3 = df.copy()
# Se agregan las labels generadas
df_birch3['cluster'] = cluster_labels
df_birch3['cluster'].value_counts()

cluster
0    5070
2    4861
1     171
Name: count, dtype: int64

----

Prueba Nro. 3

- Kmeans


In [84]:
# Se genera el experimento en mlflow
exp_name_3 = 'Experimentación clustering 3'
exp_id_3 = mlflow.create_experiment(name=exp_name_3)

In [85]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id_3, run_name="Kmeans - K=5"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=5, algorithm='elkan', max_iter=200)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 5)
    # Se registra el parámetro algortihm
    mlflow.log_param('Valor algorithm', 'elkan')
    # Se registra el parámetro max_iter
    mlflow.log_param('Valor max_iter', 200)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K5")

    # Se finaliza el registro
    mlflow.end_run()

In [86]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k5 = df.copy()
# Se agregan las labels generadas
df_kmeans_k5['cluster'] = cluster_labels
df_kmeans_k5['cluster'].value_counts()

cluster
1    3829
4    3252
0    2327
3     528
2     166
Name: count, dtype: int64

In [92]:
df_kmeans_k5.to_csv("../../../data/final/resultados_kmeans_k5.csv", sep=";", index=False)

- AgglomerativeClustering

In [87]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id_3, run_name="AGClustering - K=4"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=4, linkage='average')
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 4)
    # Se registra el parámetro linkage
    mlflow.log_param('Valor linkage', 'average')

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K4")

    # Se finaliza el registro
    mlflow.end_run()

2024/05/05 16:42:21 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [88]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k4 = df.copy()
# Se agregan las labels generadas
df_agc_k4['cluster'] = cluster_labels
df_agc_k4['cluster'].value_counts()

cluster
1    9700
0     383
3      18
2       1
Name: count, dtype: int64

In [89]:
df_agc_k4.to_csv("../../../data/final/resultados_agc_k4.csv", sep=";", index=False)

- Birch

In [90]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id_3, run_name="Birch - threshold=0.5 - n_clusters=6 - branching_factor 70"):

    # Selección y configuración de la técnica
    modelo_clusters = Birch(threshold=0.5, n_clusters=6, branching_factor=70)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro threshold
    mlflow.log_param('Valor threshold', 0.5)
    # Se registra el parámetro num clusters
    mlflow.log_param('Valor n_clusters', 6)
    # Se registra el parámetro branching_factor
    mlflow.log_param('Valor branching_factor', 70)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Birch_0.5/6/70")

    # Se finaliza el registro
    mlflow.end_run()

In [91]:
# Se preserva el conjunto de datos original para evitar problemas
df_birch6 = df.copy()
# Se agregan las labels generadas
df_birch6['cluster'] = cluster_labels
df_birch6['cluster'].value_counts()

cluster
1    3889
3    3562
5    1299
2     961
4     220
0     171
Name: count, dtype: int64